In [3]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install azure-cosmos

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install windows-tools

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Importar módulos y funciones necesarios
import os  # Módulo para interactuar con el sistema operativo
import requests  # Módulo para hacer solicitudes HTTP
import json  # Módulo para trabajar con datos JSON
import uuid  # Módulo para generar UUIDs únicos
from azure.cosmos import CosmosClient  # Cliente para interactuar con Azure Cosmos DB
import matplotlib.pyplot as plt  # Módulo para visualización de datos
from windows_tools.installed_software import get_installed_software  # Función para obtener software instalado en Windows



# Configuración de la base de datos Cosmos DB
settings = {
    'host': os.environ.get('ACCOUNT_HOST', 'https://cosmosbryan2.documents.azure.com:443/'),
    'master_key': os.environ.get('ACCOUNT_KEY', 'fpu0dLCSUoYPyhVreoO5QAwluKkfDfgfs3VsmbZSqM0GIScWKJDQB0QCEb9T5it9lwJxeL0E0bfTACDb4F0z2A=='),
    'database_id': os.environ.get('COSMOS_DATABASE', 'cosmosbryan2'),
    'container_id': os.environ.get('COSMOS_CONTAINER', 'CVE')
}

# Asignación de valores a variables internas para la configuración de Cosmos DB
HOST = settings['host']  # Obtiene el valor del host de la configuración
MASTER_KEY = settings['master_key']  # Obtiene la clave maestra de la configuración
DATABASE_ID = settings['database_id']  # Obtiene el ID de la base de datos de la configuración
CONTAINER_ID = settings['container_id']  # Obtiene el ID del contenedor de la configuración

# Función para guardar datos en Cosmos DB
def save_to_cosmos_db(software, vulnerabilities): # Crea un cliente Cosmos DB utilizando el host y la clave maestra Obtener cliente de base de datos
    # Crear cliente Cosmos DB
    client = CosmosClient(HOST, credential=MASTER_KEY) # Obtiene el cliente de la base de datos utilizando el ID de la base de datos Obtener cliente de contenedor
    # Obtener cliente de base de datos
    database = client.get_database_client(DATABASE_ID) # Obtiene el cliente del contenedor utilizando el ID del contenedor Crear objeto de información de vulnerabilidades
    # Obtener cliente de contenedor
    container = database.get_container_client(CONTAINER_ID)
    # Crear objeto de información de vulnerabilidades
    vulnerabilities_info = {
        'id': str(uuid.uuid1()),  # Genera un UUID único
        'Software': software,  # Asigna el nombre del software al campo 'Software'
        'Vulnerabilities': vulnerabilities  # Asigna la información de vulnerabilidades al campo 'Vulnerabilities' 
    }
    # Crear un nuevo item en el contenedor
    container.create_item(body=vulnerabilities_info) # Crea un nuevo documento en el contenedor de la base de datos, utilizando la 
    #información de vulnerabilidades previamente preparada




# Función para buscar vulnerabilidades
def buscar_vulnerabilidades(software):
  
    url = "https://vulners.com/api/v3/search/lucene/"
    headers = {"Content-Type": "application/json"}
    data = {
        "query": software, # Consulta de búsqueda para el software específico
        "size": 30 # Tamaño de la respuesta, es decir, la cantidad máxima de vulnerabilidades a devolver
    }
    # Realizar una solicitud POST a la API de Vulners para buscar vulnerabilidades
    response = requests.post(url, headers=headers, data=json.dumps(data))
    if response.status_code == 200:
        return response.json()  # Devuelve los datos JSON de las vulnerabilidades encontradas
    else:
        return None  # Devuelve None si la solicitud no fue exitosa
# Función para obtener detalles de CVE
def obtener_detalles_cve(cve):

    url = f'https://cve.circl.lu/api/cve/{cve}'
    # Construye la URL para obtener detalles de la vulnerabilidad utilizando la API de CVE Details
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()  # Devuelve los detalles JSON de la vulnerabilidad
    else:
        return None  # Devuelve None si la solicitud no fue exitosa
# Función para clasificar CVSS
def clasificar_cvss(cvss):
    """
    Esta función clasifica un puntaje CVSS en una categoría de riesgo.

    :param cvss: Puntaje CVSS.
    :return: Tupla con la categoría de riesgo y el puntaje CVSS.
    """
    if cvss is None:
        return "None", 0.0
    elif cvss < 4.0:
        return "Low", cvss
    elif cvss < 7.0:
        return "Medium", cvss
    elif cvss < 9.0:
        return "High", cvss
    else:
        return "Critical", cvss
# Contador de softwares con vulnerabilidades
vulnerable_software_count = 0

vulnerable_software_count = 0  # Contador para controlar la cantidad de software vulnerable encontrado

for software in get_installed_software():
    if vulnerable_software_count >= 12:  # Si ya se han encontrado 12 sofwares el programa se detiene
        break
    name = software['name']
    # Obtiene el nombre del software actual de la lista
    if name:  
        # Verifica si el nombre del software no está vacío
        first_two_words = ' '.join(name.split()[:2])
        # Obtiene las primeras dos palabras del nombre del software
        print(f"Buscando vulnerabilidades para: {first_two_words}")
        # Imprime un mensaje indicando que se están buscando vulnerabilidades para el software actual
        vulnerabilidades = buscar_vulnerabilidades(first_two_words)
        # Busca vulnerabilidades para el software actual utilizando la función buscar_vulnerabilidades
        if vulnerabilidades:
        # Verifica si se encontraron vulnerabilidades para el software actual
            software_vulnerabilities = []
            # Inicializa una lista para almacenar las vulnerabilidades del software actual
            for vulnerabilidad in vulnerabilidades['data']['search']:
                # Itera sobre las vulnerabilidades encontradas
                cve = vulnerabilidad['_source']['id']
                # Obtiene el identificador CVE de la vulnerabilidad actual
                detalles_cve = obtener_detalles_cve(cve)
                # Obtiene los detalles de la vulnerabilidad actual utilizando la función obtener_detalles_cve
                if detalles_cve:
                    # Verifica si se obtuvieron los detalles de la vulnerabilidad actual correctamente
                    cvss_classification, cvss_score = clasificar_cvss(detalles_cve['cvss'])
                    # Crear información detallada sobre la vulnerabilidad
                    info_vulnerabilidad = {
                        "CVE": cve, # Almacena el identificador CVE de la vulnerabilidad actual
                        "Summary": detalles_cve['summary'], # Almacena el resumen de la vulnerabilidad actual
                        "Published": detalles_cve['Published'],  # Almacena la fecha de publicación de la vulnerabilidad actual
                        "Last modification": detalles_cve['last-modified'], # Almacena la última modificación de la vulnerabilidad actual
                        "CVSS": cvss_score,  # Almacena el puntaje CVSS de la vulnerabilidad actual
                        "CVSS Classification": cvss_classification
                    }
                    software_vulnerabilities.append(info_vulnerabilidad)
                    # Agrega el diccionario de la vulnerabilidad actual a la lista de vulnerabilidades del software actual
            if software_vulnerabilities:
                save_to_cosmos_db(first_two_words, software_vulnerabilities)  # Guardar datos en Cosmos DB
                vulnerable_software_count += 1
                # Imprimir información de vulnerabilidades para cada dato
                print(f"Información de vulnerabilidades para {first_two_words}:")
                for vulnerability in software_vulnerabilities:
                    print("CVE Guardado")
        else:
            print(f"No se pudo obtener información sobre las vulnerabilidades de {first_two_words}.")


# Función para obtener los datos de la base de datos
def get_vulnerability_data():
    client = CosmosClient(HOST, credential=MASTER_KEY) # Crea un cliente Cosmos DB utilizando el host y la clave maestra
    database = client.get_database_client(DATABASE_ID)
    container = database.get_container_client(CONTAINER_ID)
    query = "SELECT * FROM c"
    items = list(container.query_items(query=query, enable_cross_partition_query=True))
    # Ejecuta la consulta para obtener todos los elementos del contenedor
    return items  # Devuelve la lista de elementos obtenidos de la consulta


# Llama a la función get_vulnerability_data() para obtener los datos de vulnerabilidades de la base de datos y los asigna a la variable vulnerability_data
vulnerability_data = get_vulnerability_data()

# Procesar los datos para obtener la distribución de CVSS
cvss_counts = {"None": 0, "Low": 0, "Medium": 0, "High": 0, "Critical": 0}

for data in vulnerability_data:
    vulnerabilities = data['Vulnerabilities']
    for vulnerability in vulnerabilities:
        cvss_classification = vulnerability["CVSS Classification"]
        cvss_counts[cvss_classification] += 1

# Datos para el gráfico de barras
categorias = ['Critical', 'High', 'Medium', 'Low', 'None']
valores = [cvss_counts[cat] for cat in categorias]

# Colores para los gráficos
colores = ['black', 'red', 'orange', 'green', 'blue']

# Crear gráfico de barras
plt.figure(figsize=(7, 5))
plt.bar(categorias, valores, color=colores)

# Añadir título y etiquetas a los ejes
plt.title('Impacto por criticidad de vulnerabilidades')
plt.xlabel('Criticidad')
plt.ylabel('Impacto')

# Mostrar el gráfico de barras
plt.show()

# Datos para el gráfico de tarta
etiquetas = categorias
tamaños = valores

# Crear gráfico de tarta
plt.figure(figsize=(5,5))
plt.pie(tamaños, labels=etiquetas, autopct='%1.1f%%', colors=colores, textprops={'color': '#030E00'})
plt.title('Distribución de vulnerabilidades por criticidad')

# Asegurarse de que el gráfico se muestre como un círculo
plt.axis('equal')

# Mostrar el gráfico de tarta
plt.show()

Buscando vulnerabilidades para: StarUML 6.0.1
Buscando vulnerabilidades para: Cisco Packet
Información de vulnerabilidades para Cisco Packet:
CVE Guardado
CVE Guardado
CVE Guardado
CVE Guardado
CVE Guardado
CVE Guardado
CVE Guardado
Buscando vulnerabilidades para: Git
Buscando vulnerabilidades para: MATLAB R2023a
Buscando vulnerabilidades para: Microsoft SQL
Información de vulnerabilidades para Microsoft SQL:
CVE Guardado
CVE Guardado
CVE Guardado
Buscando vulnerabilidades para: Microsoft SQL
Información de vulnerabilidades para Microsoft SQL:
CVE Guardado
CVE Guardado
CVE Guardado
Buscando vulnerabilidades para: Aplicaciones de
Buscando vulnerabilidades para: Microsoft OneDrive
Información de vulnerabilidades para Microsoft OneDrive:
CVE Guardado
CVE Guardado
CVE Guardado
Buscando vulnerabilidades para: SQLyog 11.11
Buscando vulnerabilidades para: WinRAR 6.21
Buscando vulnerabilidades para: Microsoft .NET
Información de vulnerabilidades para Microsoft .NET:
CVE Guardado
CVE Guardado
C